# Library of Plans

In [0]:
from bluesky import RunEngine
from bluesky.callbacks import LiveTable
from bluesky.examples import motor, det


RE = RunEngine({'owner': 'demo', 'group': 'demo', 'beamline_id': 'demo'})

## How to use a plan

We'll start with a simple plan which moves nothing but takes one measurement (a "count") from a list of detectors.

In [0]:
from bluesky.plans import Count

In [0]:
count = Count([det])
RE(count, LiveTable([det]))

The `count` object encapsulates a list of messages.

In [0]:
list(count)

We can edit the attributes of count, causing that list to change.

In [0]:
count.num

In [0]:
len(list(count))  # how many messages does it issue?

In [0]:
count.num = 2
len(list(count))  # how about now?

In [0]:
RE(count, LiveTable([det]))

It might be convenient to have more than one instance of `Count` at our disposal.

In [0]:
count_to_ten = Count([det], num=10)

## 1D Scans

The most common 1D step scan takes linearly spaced steps. This is what `AbsScanPlan` and `DeltaScanPlan` do.

In [0]:
from bluesky.plans import AbsScanPlan, DeltaScanPlan, AbsListScanPlan, DeltaListScanPlan

In [0]:
RE(AbsScanPlan([det], motor, 1, 5, 5), LiveTable([motor, det]))

In [0]:
motor.set(10)  # move the motor to 10; this is where the relative motion of the "DeltaScanPlan" will start

In [0]:
RE(DeltaScanPlan([det], motor, 1, 5, 5), LiveTable([motor, det]))

The steps need not be linearly spaced; we can specify a list of positions manually.

In [0]:
RE(AbsListScanPlan([det], motor, [1, 3, 7, 15]), LiveTable([motor, det]))

There are also log-spaced options `LogAbsScanPlan` and `LogDeltaScanPlan`.

## Scans that Move Motors in Parallel

We call these "inner product" scans because the overall trajectory is in a sense an inner product of the individual motors' trajectories.

In [0]:
from bluesky.plans import InnerProductAbsScanPlan, InnerProductDeltaScanPlan
from bluesky.examples import motor1, motor2
# Turn of "fake sleeping" that simulates motors moving.
motor1._fake_sleep = 0
motor2._fake_sleep = 0

Let's plan a 5-point trajectory moving motor1 from 1 to 5 and motor2 for 10 to 50 in linearly spaced steps.

In [0]:
plan = InnerProductAbsScanPlan([det], 5, motor1, 1, 5, motor2, 10, 50)
RE(plan, LiveTable(['motor1', 'motor2']))

We are not limited to two motors. This works for any number.

## Scans that Move Motors in a Mesh

We call mesh scans "output product" scans because the overall trajectory is in a sense an outer product of the individual motors' trajectories.

In [0]:
from bluesky.plans import OuterProductAbsScanPlan, OuterProductDeltaScanPlan

Let's move motor1 in 5 steps from 1 to 5 and motor2 in 3 steps from 10 to 3. Unlike the inner product scans, each motor can take different number of steps, so we specify them separately.

In [0]:
plan = OuterProductAbsScanPlan([det], motor1, 1, 5, 5, motor2, 10, 30, 3, False)  # We explain the 'False' argument later.
RE(plan, LiveTable(['motor1', 'motor2']))

Notice that the first motor is the "slow" motor and the second motor is the "fast" motor. By choosing the order the motors are given, you choose the path that the mesh is traversed.

In addition to start, stop, and number of steps, every motor except the slowest motor gets a True or False argument that indicates where it should take a snake-like trajectory for maximum efficiency. Observe the difference if we set motor's `snake` argument to `True`. (It was `False` above.)

In [0]:
plan = OuterProductAbsScanPlan([det], motor1, 1, 5, 5, motor2, 10, 30, 3, True)
RE(plan, LiveTable(['motor1', 'motor2']))

## Generalized ND Scans

What about a scan that moves two motors together (an inner product) in a mesh (outer product) against a third motor? To define arbitrary N-dimensional compositions of trajectories, we use `cycler`. Adding two cyclers makes an inner product; multiplying two cyclers makes an outer product.

In [0]:
from cycler import cycler

In [0]:
list(cycler(motor1, [1, 2, 3]) + cycler(motor2, [10, 20, 30]))

In [0]:
list(cycler(motor1, [1, 2, 3]) * cycler(motor2, [10, 20, 30]))

The ScanND object accepts a cycler and makes it into a plan.

In [0]:
from bluesky import PlanND
from bluesky.examples import motor3
motor3._fake_sleep = 0 

inner = cycler(motor1, [1, 2, 3]) + cycler(motor2, [10, 20, 30])
plan = PlanND([det], inner)
RE(plan, LiveTable([motor1, motor2]))

In [0]:
plan = PlanND([det], inner * cycler(motor3, [4, 5, 6]))
RE(plan, LiveTable([motor1, motor2, motor3]))

## Adaptive Scans

In [0]:
from bluesky.plans import AdaptiveAbsScanPlan

# minimum step size 0.01
# maximum step size 1
# desired delta between steps 0.05
# backward motion is allowed (True)

ad_scan = AdaptiveAbsScanPlan([det], 'det', motor, -15, 10, .01, 5, .05, True)

# We'll include a plot.
%matplotlib notebook
from bluesky.callbacks import LivePlot

RE(ad_scan, [LiveTable(['det', 'motor']), LivePlot('det', 'motor', markersize=10, marker='o')])